# Make Predictions 

* On unseen data.

In [1]:
def go_up_from_current_directory(*, go_up: int = 1) -> None:
    """This is used to up a number of directories.
    Args:
    -----
    go_up: int, default=1
        This indicates the number of times to go back up from the current directory.
    Returns:
    --------
    None
    """
    import os
    import sys

    CONST: str = "../"
    NUM: str = CONST * go_up

    # Goto the previous directory
    prev_directory = os.path.join(os.path.dirname(__name__), NUM)
    # Get the 'absolute path' of the previous directory
    abs_path_prev_directory = os.path.abspath(prev_directory)

    # Add the path to the System paths
    sys.path.insert(0, abs_path_prev_directory)
    print(abs_path_prev_directory)

In [2]:
# Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# for saving the pipeline
import joblib

# Custom Module(s)
go_up_from_current_directory(go_up=1)
import preprocessor as pp
import utilities
from lending_data import LendingData

# Built-in library
import itertools
import re
import json
import typing as tp

# pandas settings
pd.options.display.max_rows = 1_000
pd.options.display.max_columns = 1_000
pd.options.display.max_colwidth = 600

# Black code formatter (Optional)
%load_ext lab_black
# auto reload imports
%load_ext autoreload
%autoreload 2

/Users/neidu/Desktop/Projects/Personal/tutorials/LendingClub Issued Loans


In [3]:
fp = "../data/orig_test_data.csv"  # Filepath

orig_test_data = utilities.load_data(filepath=fp, format_="csv", low_memory=False)

orig_test_data.head()

The shape of the data: (1000, 72)



,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,79121451,NaN,23025,23025,23025.0,60 months,20.99,622.78,E,E1,Licensed Plumber,2 years,RENT,65000.0,Source Verified,Jun-2016,Late (31-120 days),n,NaN,debt_consolidation,Debt consolidation,802xx,CO,13.07,0,Sep-2000,2.0,30.0,87.0,10,1,10895.0,38.9,16,f,19293.99,19293.99,9339.520000,9339.52,3731.01,5608.51,0.0,0.0,0.0,Sep-2017,622.78,Jan-2018,Dec-2017,0,30.0,1,Individual,NaN,NaN,NaN,0,0.0,19991.0,1.0,0.0,1.0,13.0,9096.0,70.0,3.0,7.0,5284.0,49.0,28000.0,2.0,3.0,3.0
1,114207950,NaN,9450,9450,9450.0,36 months,13.59,321.10,C,C2,NaN,NaN,OWN,25635.0,Verified,Aug-2017,Current,n,NaN,debt_consolidation,Debt consolidation,761xx,TX,31.55,0,Apr-1999,0.0,33.0,NaN,13,0,15253.0,43.7,35,f,8579.03,8579.03,1277.270000,1277.27,870.97,406.30,0.0,0.0,0.0,Dec-2017,321.10,Jan-2018,Dec-2017,0,33.0,1,Individual,NaN,NaN,NaN,0,0.0,113213.0,1.0,0.0,0.0,58.0,0.0,NaN,2.0,10.0,9613.0,44.0,36000.0,0.0,0.0,0.0
2,92369915,NaN,19200,19200,19200.0,36 months,13.49,651.47,C,C2,NaN,NaN,MORTGAGE,40000.0,Verified,Nov-2016,Current,n,NaN,home_improvement,Home improvement,335xx,FL,8.01,3,Nov-1995,0.0,15.0,15.0,9,2,7835.0,41.5,17,w,13138.63,13138.63,8454.720000,8454.72,6061.37,2393.35,0.0,0.0,0.0,Dec-2017,651.47,Jan-2018,Dec-2017,0,20.0,1,Individual,NaN,NaN,NaN,0,0.0,154027.0,2.0,0.0,0.0,113.0,0.0,NaN,1.0,1.0,3407.0,42.0,18900.0,1.0,0.0,2.0
3,99965324,NaN,6000,6000,6000.0,36 months,7.24,185.93,A,A3,Claims Adjustor,5 years,MORTGAGE,65000.0,Not Verified,Mar-2017,Fully Paid,n,NaN,home_improvement,Home improvement,123xx,NY,4.86,0,Feb-2012,0.0,NaN,NaN,9,0,3819.0,15.7,13,w,0.00,0.00,6002.030533,6002.03,6000.00,2.03,0.0,0.0,0.0,Apr-2017,6004.44,NaN,Dec-2017,0,NaN,1,Individual,NaN,NaN,NaN,0,0.0,154027.0,0.0,1.0,3.0,12.0,7286.0,79.0,2.0,4.0,3790.0,33.0,24300.0,3.0,1.0,2.0
4,68053146,NaN,15000,15000,15000.0,36 months,7.91,469.43,A,A5,Charge Nurse,9 years,OWN,75000.0,Not Verified,Jan-2016,Current,n,NaN,credit_card,Credit card refinancing,074xx,NJ,8.76,0,Feb-1984,0.0,NaN,NaN,13,0,14255.0,56.8,24,w,5829.79,5829.79,10790.300000,10790.30,9170.21,1620.09,0.0,0.0,0.0,Dec-2017,469.43,Jan-2018,Dec-2017,0,NaN,1,Individual,NaN,NaN,NaN,0,0.0,88917.0,1.0,0.0,0.0,65.0,0.0,NaN,2.0,2.0,6514.0,57.0,25100.0,0.0,0.0,0.0


In [4]:
num_vars_to_drop = [
    LendingData.ANNUAL_INC_JOINT,
    LendingData.COLLECTION_RECOVERY_FEE,
    LendingData.DTI_JOINT,
    LendingData.FUNDED_AMNT,
    LendingData.FUNDED_AMNT_INV,
    LendingData.ID,
    LendingData.INSTALLMENT,
    LendingData.MEMBER_ID,
    LendingData.MTHS_SINCE_LAST_RECORD,
    LendingData.OPEN_RV_24M,
    LendingData.OUT_PRNCP_INV,
    LendingData.TOTAL_PYMNT_INV,
    LendingData.TOTAL_REC_PRNCP,
]

cat_vars_to_drop = [
    LendingData.DESC,
    LendingData.EARLIEST_CR_LINE,
    LendingData.EMP_TITLE,
    LendingData.ISSUE_D,
    LendingData.LAST_CREDIT_PULL_D,
    LendingData.LAST_PYMNT_D,
    LendingData.NEXT_PYMNT_D,
    LendingData.POLICY_CODE,
    LendingData.PYMNT_PLAN,
    LendingData.SUB_GRADE,
    LendingData.TITLE,
    LendingData.VERIFICATION_STATUS_JOINT,
]

# Drop the variables
orig_test_data.drop(columns=num_vars_to_drop + cat_vars_to_drop, inplace=True)

In [5]:
numerical_vars_with_na = [
    LendingData.ALL_UTIL,
    LendingData.DTI,
    LendingData.IL_UTIL,
    LendingData.INQ_FI,
    LendingData.INQ_LAST_12M,
    LendingData.INQ_LAST_6MTHS,
    LendingData.MAX_BAL_BC,
    LendingData.MTHS_SINCE_LAST_DELINQ,
    LendingData.MTHS_SINCE_LAST_MAJOR_DEROG,
    LendingData.MTHS_SINCE_RCNT_IL,
    LendingData.OPEN_ACC_6M,
    LendingData.OPEN_IL_12M,
    LendingData.OPEN_IL_24M,
    LendingData.OPEN_RV_12M,
    LendingData.REVOL_UTIL,
    LendingData.TOTAL_BAL_IL,
    LendingData.TOTAL_CU_TL,
]
# Categorical variables with NA in train set
categorical_vars_with_na_frequent = [LendingData.EMP_LENGTH, LendingData.ZIP_CODE]

# Variables with NaNs in the training data
vars_with_na_train_data = numerical_vars_with_na + categorical_vars_with_na_frequent
NA_THRESH = 0

# Check the NaN values in the test data
vars_with_na_test_data = [
    var
    for var in orig_test_data.columns
    if orig_test_data[var].isna().sum() > NA_THRESH
]
vars_with_na_test_data

['emp_length',
 'mths_since_last_delinq',
 'mths_since_last_major_derog',
 'mths_since_rcnt_il',
 'il_util']

In [6]:
# Variables with NaN values in ONLY the test data.
vars_with_na_test_ONLY = [
    *set(vars_with_na_test_data).difference(set(vars_with_na_train_data))
]
vars_with_na_test_ONLY

[]

In [7]:
orig_test_data[vars_with_na_test_ONLY].isna().mean().mul(100)

Series([], dtype: float64)

In [8]:
# Drop the NaNs occurring ONLY in the test data.
target = LendingData.LOAN_STATUS

test_data = orig_test_data.drop(columns=[target])
test_data.dropna(subset=vars_with_na_test_ONLY, inplace=True)

In [9]:
"CATEGORICAL_VARS".lower()

'categorical_vars'

In [10]:
numerical_vars = [
    LendingData.ACC_NOW_DELINQ,
    LendingData.ALL_UTIL,
    LendingData.ANNUAL_INC,
    LendingData.COLLECTIONS_12_MTHS_EX_MED,
    LendingData.DELINQ_2YRS,
    LendingData.DTI,
    LendingData.IL_UTIL,
    LendingData.INQ_FI,
    LendingData.INQ_LAST_12M,
    LendingData.INQ_LAST_6MTHS,
    LendingData.INT_RATE,
    LendingData.LAST_PYMNT_AMNT,
    LendingData.LOAN_AMNT,
    LendingData.MAX_BAL_BC,
    LendingData.MTHS_SINCE_LAST_DELINQ,
    LendingData.MTHS_SINCE_LAST_MAJOR_DEROG,
    LendingData.MTHS_SINCE_RCNT_IL,
    LendingData.OPEN_ACC,
    LendingData.OPEN_ACC_6M,
    LendingData.OPEN_IL_12M,
    LendingData.OPEN_IL_24M,
    LendingData.OPEN_RV_12M,
    LendingData.OUT_PRNCP,
    LendingData.PUB_REC,
    LendingData.RECOVERIES,
    LendingData.REVOL_BAL,
    LendingData.REVOL_UTIL,
    LendingData.TOT_COLL_AMT,
    LendingData.TOT_CUR_BAL,
    LendingData.TOTAL_ACC,
    LendingData.TOTAL_BAL_IL,
    LendingData.TOTAL_CU_TL,
    LendingData.TOTAL_PYMNT,
    LendingData.TOTAL_REC_INT,
    LendingData.TOTAL_REC_LATE_FEE,
    LendingData.TOTAL_REV_HI_LIM,
]
categorical_vars = [
    LendingData.ADDR_STATE,
    LendingData.APPLICATION_TYPE,
    LendingData.EMP_LENGTH,
    LendingData.GRADE,
    LendingData.HOME_OWNERSHIP,
    LendingData.INITIAL_LIST_STATUS,
    LendingData.PURPOSE,
    LendingData.TERM,
    LendingData.VERIFICATION_STATUS,
    LendingData.ZIP_CODE,
]

features = numerical_vars + categorical_vars

# Select the relevant features
final_test_data = test_data[features]

# Save data
final_test_data.to_csv("../data/test.csv", index=False)

In [11]:
# Persit the model
fp = "../models/logistic_pipe.joblib"

with open(fp, "rb") as file:
    logistic_pipe = joblib.load(file)

In [12]:
logistic_pipe.named_steps

{'missing_indicator': AddMissingIndicator(variables=['all_util', 'dti', 'il_util', 'inq_fi',
                                'inq_last_12m', 'inq_last_6mths', 'max_bal_bc',
                                'mths_since_last_delinq',
                                'mths_since_last_major_derog',
                                'mths_since_rcnt_il', 'open_acc_6m',
                                'open_il_12m', 'open_il_24m', 'open_rv_12m',
                                'revol_util', 'total_bal_il', 'total_cu_tl']),
 'median_imputation': MeanMedianImputer(variables=['all_util', 'dti', 'inq_last_6mths',
                              'mths_since_last_delinq',
                              'mths_since_last_major_derog']),
 'mean_imputation': MeanMedianImputer(imputation_method='mean',
                   variables=['il_util', 'inq_fi', 'inq_last_12m', 'max_bal_bc',
                              'mths_since_rcnt_il', 'open_acc_6m', 'open_il_12m',
                              'open_il_24m', 'o

In [13]:
final_test_data[features].isna().mean().mul(100)

acc_now_delinq                  0.0
all_util                        0.0
annual_inc                      0.0
collections_12_mths_ex_med      0.0
delinq_2yrs                     0.0
dti                             0.0
il_util                        12.8
inq_fi                          0.0
inq_last_12m                    0.0
inq_last_6mths                  0.0
int_rate                        0.0
last_pymnt_amnt                 0.0
loan_amnt                       0.0
max_bal_bc                      0.0
mths_since_last_delinq         49.0
mths_since_last_major_derog    71.5
mths_since_rcnt_il              2.7
open_acc                        0.0
open_acc_6m                     0.0
open_il_12m                     0.0
open_il_24m                     0.0
open_rv_12m                     0.0
out_prncp                       0.0
pub_rec                         0.0
recoveries                      0.0
revol_bal                       0.0
revol_util                      0.0
tot_coll_amt                

In [ ]:
pred = logistic_pipe.predict(final_test_data)
pred_proba = logistic_pipe.predict_proba(final_test_data)[:, 1]
result = itertools.zip_longest(pred, pred_proba)

df = pd.DataFrame(result, columns=["Default", "Predict_probability"])
df["Predict_probability"] = round(df["Predict_probability"], 3)

df